# Paper D: Point-Dependent Directional Weighting Experiments

## 点依存形式による曲率発生の検証

### 理論的背景

式(6)を点依存形式に拡張：

$$D_w^{\text{aniso}}(P, Q) = D(P, Q) + \beta|w| \cdot h(\theta_Q) \cdot (u^T(\theta_P - \theta_Q))^2$$

where $h(\theta') = 1 + \gamma(v^T \theta')^2$

この形式により、3階混合微分が非ゼロとなり、曲率が発生する。

### 実験内容
1. 2Dガウス分布
2. カテゴリカル分布（自然パラメータ θ = log p を使用）
3. 曲率関連量の数値確認

---

**Runtime**: ~1 minute (CPU only)

---

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

import os
SAVE_DIR = '/content/drive/MyDrive/paper-D/experiments'
os.makedirs(SAVE_DIR, exist_ok=True)
print(f'Results will be saved to: {SAVE_DIR}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from typing import Tuple, Callable
import json
import csv
from datetime import datetime

# 再現性のための乱数シード
np.random.seed(42)

# Plot Settings (Academic Journal Style)
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 10
rcParams['axes.labelsize'] = 11
rcParams['axes.titlesize'] = 11
rcParams['xtick.labelsize'] = 9
rcParams['ytick.labelsize'] = 9
rcParams['legend.fontsize'] = 9
rcParams['figure.dpi'] = 150
rcParams['savefig.dpi'] = 300
rcParams['savefig.bbox'] = 'tight'
rcParams['axes.linewidth'] = 0.8

print("Paper D: Point-Dependent Directional Weighting Experiments")
print(f"Execution date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"NumPy version: {np.__version__}")

## Part 1: 2D Gaussian Manifold

In [ ]:
# ============================================
# 2D Gaussian: 基本設定
# ============================================

# Pythagorean triple (from v2 experiments)
# P, Q, R form a right triangle with right angle at Q
P_gauss = np.array([0.0, 1.5])
Q_gauss = np.array([0.0, 0.0])
R_gauss = np.array([2.0, 0.0])

# For 2D Gaussian N(μ, I), natural parameter θ = μ
theta_P = P_gauss
theta_Q = Q_gauss
theta_R = R_gauss

print("2D Gaussian Pythagorean Triple:")
print(f"  P (θ_P) = {theta_P}")
print(f"  Q (θ_Q) = {theta_Q}")
print(f"  R (θ_R) = {theta_R}")

In [ ]:
# ============================================
# 2D Gaussian: KL Divergence
# ============================================

def kl_gaussian(theta_p: np.ndarray, theta_q: np.ndarray) -> float:
    """
    KL divergence for 2D Gaussian N(μ, I)
    D(P || Q) = 0.5 * ||μ_P - μ_Q||^2
    """
    diff = theta_p - theta_q
    return 0.5 * np.dot(diff, diff)

# Verify baseline Pythagorean theorem
D_PQ = kl_gaussian(theta_P, theta_Q)
D_QR = kl_gaussian(theta_Q, theta_R)
D_PR = kl_gaussian(theta_P, theta_R)

epsilon_baseline = abs(D_PR - D_PQ - D_QR)

print("Baseline Pythagorean Check (2D Gaussian):")
print(f"  D(P, Q) = {D_PQ}")
print(f"  D(Q, R) = {D_QR}")
print(f"  D(P, R) = {D_PR}")
print(f"  D(P,R) - D(P,Q) - D(Q,R) = {D_PR - D_PQ - D_QR}")
print(f"  ε(0) = {epsilon_baseline:.2e}")
print(f"  ✓ Pythagorean theorem holds" if epsilon_baseline < 1e-14 else "  ✗ Error!")

In [ ]:
# ============================================
# 2D Gaussian: Point-Dependent Perturbation
# ============================================

def h_function(theta_prime: np.ndarray, v: np.ndarray, gamma: float) -> float:
    """
    Point-dependent coefficient: h(θ') = 1 + γ(v^T θ')^2
    """
    return 1.0 + gamma * (np.dot(v, theta_prime))**2

def dh_dtheta(theta_prime: np.ndarray, v: np.ndarray, gamma: float) -> np.ndarray:
    """
    Gradient of h: ∂h/∂θ' = 2γ(v^T θ')v
    """
    return 2.0 * gamma * np.dot(v, theta_prime) * v

def D_w_aniso_gaussian(theta_p: np.ndarray, theta_q: np.ndarray,
                       u: np.ndarray, v: np.ndarray,
                       beta: float, gamma: float, w_mag: float) -> float:
    """
    Point-dependent anisotropic divergence for Gaussian:
    D_w(P, Q) = D(P, Q) + β|w| h(θ_Q) (u^T(θ_P - θ_Q))^2
    """
    base_div = kl_gaussian(theta_p, theta_q)
    h_val = h_function(theta_q, v, gamma)
    diff = theta_p - theta_q
    aniso_term = beta * w_mag * h_val * (np.dot(u, diff))**2
    return base_div + aniso_term

def D_w_iso_gaussian(theta_p: np.ndarray, theta_q: np.ndarray,
                     alpha: float, w_mag: float) -> float:
    """
    Isotropic divergence:
    D_w^iso(P, Q) = (1 + α|w|^2) D(P, Q)
    """
    base_div = kl_gaussian(theta_p, theta_q)
    return (1.0 + alpha * w_mag**2) * base_div

print("Point-dependent perturbation functions defined.")

In [ ]:
# ============================================
# 2D Gaussian: Experiment Parameters
# ============================================

# Direction vectors
u_gauss = np.array([1.0, 1.0]) / np.sqrt(2)  # Anisotropic direction
v_gauss = np.array([1.0, -1.0]) / np.sqrt(2)  # Point-dependence direction

# Parameters
beta_gauss = 1.0   # Coupling constant
gamma_gauss = 0.5  # Point-dependence strength
alpha_gauss = 0.5  # Isotropic scaling

# |w| range
w_values = np.linspace(0, 2, 41)

print("2D Gaussian Experiment Parameters:")
print(f"  u = {u_gauss} (anisotropic direction)")
print(f"  v = {v_gauss} (point-dependence direction)")
print(f"  β = {beta_gauss}")
print(f"  γ = {gamma_gauss}")
print(f"  α = {alpha_gauss} (for isotropic)")
print(f"  |w| range: [{w_values[0]}, {w_values[-1]}], {len(w_values)} points")

In [ ]:
# ============================================
# 2D Gaussian: Compute Pythagorean Residuals
# ============================================

epsilon_aniso_gauss = []
epsilon_iso_gauss = []

for w_mag in w_values:
    # Anisotropic (point-dependent)
    D_w_PQ = D_w_aniso_gaussian(theta_P, theta_Q, u_gauss, v_gauss, beta_gauss, gamma_gauss, w_mag)
    D_w_QR = D_w_aniso_gaussian(theta_Q, theta_R, u_gauss, v_gauss, beta_gauss, gamma_gauss, w_mag)
    D_w_PR = D_w_aniso_gaussian(theta_P, theta_R, u_gauss, v_gauss, beta_gauss, gamma_gauss, w_mag)
    eps_aniso = abs(D_w_PR - D_w_PQ - D_w_QR)
    epsilon_aniso_gauss.append(eps_aniso)
    
    # Isotropic
    D_w_PQ_iso = D_w_iso_gaussian(theta_P, theta_Q, alpha_gauss, w_mag)
    D_w_QR_iso = D_w_iso_gaussian(theta_Q, theta_R, alpha_gauss, w_mag)
    D_w_PR_iso = D_w_iso_gaussian(theta_P, theta_R, alpha_gauss, w_mag)
    eps_iso = abs(D_w_PR_iso - D_w_PQ_iso - D_w_QR_iso)
    epsilon_iso_gauss.append(eps_iso)

epsilon_aniso_gauss = np.array(epsilon_aniso_gauss)
epsilon_iso_gauss = np.array(epsilon_iso_gauss)

print("2D Gaussian Results:")
print(f"  ε_aniso(0) = {epsilon_aniso_gauss[0]:.2e}")
print(f"  ε_aniso(max) = {epsilon_aniso_gauss[-1]:.6f}")
print(f"  ε_iso(0) = {epsilon_iso_gauss[0]:.2e}")
print(f"  ε_iso(max) = {epsilon_iso_gauss[-1]:.2e}")

In [ ]:
# ============================================
# 2D Gaussian: Verify Third-Order Mixed Derivative
# ============================================

def verify_third_derivative_gaussian(theta_q: np.ndarray, u: np.ndarray, v: np.ndarray,
                                      beta: float, gamma: float, w_mag: float) -> np.ndarray:
    """
    Compute the connection perturbation δΓ_{ij,k} for the point-dependent form.
    
    For B(θ, θ') = h(θ') (u^T(θ - θ'))^2:
    ∂_i ∂_j B = h(θ') u_i u_j · 2
    ∂_i ∂_j ∂'_k B = 2 u_i u_j ∂_k h(θ')
    
    At θ = θ':
    δΓ_{ij,k} = -β|w| · 2 u_i u_j ∂_k h(θ)
    """
    dh = dh_dtheta(theta_q, v, gamma)  # ∂h/∂θ' evaluated at θ_Q
    
    # δΓ_{ij,k} is a rank-3 tensor
    n = len(theta_q)
    delta_Gamma = np.zeros((n, n, n))
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                delta_Gamma[i, j, k] = -beta * w_mag * 2 * u[i] * u[j] * dh[k]
    
    return delta_Gamma

# Compute at θ_Q with |w| = 1
delta_Gamma_Q = verify_third_derivative_gaussian(theta_Q, u_gauss, v_gauss, beta_gauss, gamma_gauss, 1.0)

print("Third-Order Mixed Derivative Verification (at θ_Q, |w|=1):")
print(f"  ∂h/∂θ' at θ_Q = {dh_dtheta(theta_Q, v_gauss, gamma_gauss)}")
print(f"  Note: h(θ_Q) = h(0,0) = 1 + γ(v^T · 0)^2 = 1")
print(f"  But ∂h/∂θ' = 2γ(v^T θ')v = 0 at θ'=0")
print("")
print("  → At θ_Q = (0,0), the gradient vanishes.")
print("  → Need to evaluate at a non-zero point for non-trivial δΓ.")
print("")

# Compute at θ_R (non-zero point)
delta_Gamma_R = verify_third_derivative_gaussian(theta_R, u_gauss, v_gauss, beta_gauss, gamma_gauss, 1.0)

print("Third-Order Mixed Derivative at θ_R = (2, 0):")
print(f"  h(θ_R) = {h_function(theta_R, v_gauss, gamma_gauss):.4f}")
print(f"  ∂h/∂θ' at θ_R = {dh_dtheta(theta_R, v_gauss, gamma_gauss)}")
print(f"  δΓ tensor (non-zero components):")
for i in range(2):
    for j in range(2):
        for k in range(2):
            if abs(delta_Gamma_R[i,j,k]) > 1e-10:
                print(f"    δΓ[{i},{j},{k}] = {delta_Gamma_R[i,j,k]:.6f}")

In [ ]:
# ============================================
# 2D Gaussian: Plot Results
# ============================================

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Main plot
ax1 = axes[0]
ax1.plot(w_values, epsilon_aniso_gauss, 'b-', linewidth=2, label='Anisotropic (point-dependent)')
ax1.plot(w_values, epsilon_iso_gauss, 'r--', linewidth=2, label='Isotropic')
ax1.set_xlabel('|w|', fontsize=12)
ax1.set_ylabel('Pythagorean residual ε(w)', fontsize=12)
ax1.set_title('2D Gaussian: Pythagorean Residual vs |w|', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 2])
ax1.set_ylim(bottom=0)

# Right: Log scale for isotropic (to show machine precision)
ax2 = axes[1]
ax2.semilogy(w_values, epsilon_aniso_gauss + 1e-20, 'b-', linewidth=2, label='Anisotropic')
ax2.semilogy(w_values, epsilon_iso_gauss + 1e-20, 'r--', linewidth=2, label='Isotropic')
ax2.axhline(y=1e-15, color='gray', linestyle=':', label='Machine precision')
ax2.set_xlabel('|w|', fontsize=12)
ax2.set_ylabel('ε(w) [log scale]', fontsize=12)
ax2.set_title('2D Gaussian: Log Scale', fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_xlim([0, 2])

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/D_gaussian_point_dependent.png', dpi=150, bbox_inches='tight')
plt.savefig(f'{SAVE_DIR}/D_gaussian_point_dependent.pdf', bbox_inches='tight')
plt.show()

print(f"Figures saved to: {SAVE_DIR}/D_gaussian_point_dependent.png/pdf")

## Part 2: Categorical Distribution

In [ ]:
# ============================================
# Categorical: Basic Setup
# ============================================

# Pythagorean triple (from v2 experiments, satisfies orthogonality condition)
P_cat = np.array([0.30849625, 0.09150375, 0.35, 0.25])
Q_cat = np.array([0.25, 0.25, 0.25, 0.25])  # Uniform
R_cat = np.array([0.1, 0.2, 0.3, 0.4])

# Natural parameters: θ = log p
# Note: These have gauge freedom (can add constant to all components)
theta_P_cat = np.log(P_cat)
theta_Q_cat = np.log(Q_cat)
theta_R_cat = np.log(R_cat)

print("Categorical Distribution Pythagorean Triple:")
print(f"  P = {P_cat}")
print(f"  Q = {Q_cat}")
print(f"  R = {R_cat}")
print("")
print("Natural Parameters (θ = log p):")
print(f"  θ_P = {theta_P_cat}")
print(f"  θ_Q = {theta_Q_cat}")
print(f"  θ_R = {theta_R_cat}")

In [ ]:
# ============================================
# Categorical: KL Divergence
# ============================================

def kl_categorical(p: np.ndarray, q: np.ndarray) -> float:
    """
    KL divergence for categorical distributions
    D(P || Q) = Σ p_i log(p_i / q_i)
    """
    return np.sum(p * np.log(p / q))

# Verify orthogonality condition: Σ(p_i - q_i) log(r_i / q_i) = 0
orthog_check = np.sum((P_cat - Q_cat) * np.log(R_cat / Q_cat))
print(f"Orthogonality condition: Σ(p_i - q_i) log(r_i / q_i) = {orthog_check:.2e}")

# Verify baseline Pythagorean theorem
D_PQ_cat = kl_categorical(P_cat, Q_cat)
D_QR_cat = kl_categorical(Q_cat, R_cat)
D_PR_cat = kl_categorical(P_cat, R_cat)

epsilon_baseline_cat = abs(D_PR_cat - D_PQ_cat - D_QR_cat)

print("")
print("Baseline Pythagorean Check (Categorical):")
print(f"  D(P, Q) = {D_PQ_cat:.6f}")
print(f"  D(Q, R) = {D_QR_cat:.6f}")
print(f"  D(P, R) = {D_PR_cat:.6f}")
print(f"  ε(0) = {epsilon_baseline_cat:.2e}")

In [ ]:
# ============================================
# Categorical: Point-Dependent Perturbation
# ============================================

def D_w_aniso_categorical(p: np.ndarray, q: np.ndarray,
                          u: np.ndarray, v: np.ndarray,
                          beta: float, gamma: float, w_mag: float) -> float:
    """
    Point-dependent anisotropic divergence for categorical:
    D_w(P, Q) = D(P, Q) + β|w| h(θ_Q) (u^T(θ_P - θ_Q))^2
    
    where θ = log p (natural parameters)
    and h(θ') = 1 + γ(v^T θ')^2
    
    Note: u must be tangent (Σu_i = 0) for gauge invariance
    """
    # Base divergence
    base_div = kl_categorical(p, q)
    
    # Natural parameters
    theta_p = np.log(p)
    theta_q = np.log(q)
    
    # Point-dependent coefficient
    h_val = 1.0 + gamma * (np.dot(v, theta_q))**2
    
    # Anisotropic term (in natural coordinates)
    theta_diff = theta_p - theta_q
    aniso_term = beta * w_mag * h_val * (np.dot(u, theta_diff))**2
    
    return base_div + aniso_term

def D_w_iso_categorical(p: np.ndarray, q: np.ndarray,
                        alpha: float, w_mag: float) -> float:
    """
    Isotropic divergence:
    D_w^iso(P, Q) = (1 + α|w|^2) D(P, Q)
    """
    base_div = kl_categorical(p, q)
    return (1.0 + alpha * w_mag**2) * base_div

print("Point-dependent perturbation functions for categorical defined.")

In [ ]:
# ============================================
# Categorical: Experiment Parameters
# ============================================

# Direction vectors (must be tangent: Σu_i = 0 for gauge invariance)
# Construct orthonormal tangent vectors on the simplex
u_cat_raw = np.array([1.0, -1.0, 1.0, -1.0])
u_cat = u_cat_raw / np.linalg.norm(u_cat_raw)

v_cat_raw = np.array([1.0, 1.0, -1.0, -1.0])
v_cat = v_cat_raw / np.linalg.norm(v_cat_raw)

# Verify tangent condition
print("Tangent condition verification:")
print(f"  Σu_i = {np.sum(u_cat):.2e} (should be 0)")
print(f"  Σv_i = {np.sum(v_cat):.2e} (should be 0)")

# Parameters
beta_cat = 0.5   # Coupling constant (adjusted for simplex scale)
gamma_cat = 0.1  # Point-dependence strength
alpha_cat = 0.5  # Isotropic scaling

# |w| range
w_values_cat = np.linspace(0, 1, 41)

print("")
print("Categorical Experiment Parameters:")
print(f"  u = {u_cat}")
print(f"  v = {v_cat}")
print(f"  β = {beta_cat}")
print(f"  γ = {gamma_cat}")
print(f"  α = {alpha_cat}")
print(f"  |w| range: [{w_values_cat[0]}, {w_values_cat[-1]}]")

In [ ]:
# ============================================
# Categorical: Compute Pythagorean Residuals
# ============================================

epsilon_aniso_cat = []
epsilon_iso_cat = []

for w_mag in w_values_cat:
    # Anisotropic (point-dependent)
    D_w_PQ = D_w_aniso_categorical(P_cat, Q_cat, u_cat, v_cat, beta_cat, gamma_cat, w_mag)
    D_w_QR = D_w_aniso_categorical(Q_cat, R_cat, u_cat, v_cat, beta_cat, gamma_cat, w_mag)
    D_w_PR = D_w_aniso_categorical(P_cat, R_cat, u_cat, v_cat, beta_cat, gamma_cat, w_mag)
    eps_aniso = abs(D_w_PR - D_w_PQ - D_w_QR)
    epsilon_aniso_cat.append(eps_aniso)
    
    # Isotropic
    D_w_PQ_iso = D_w_iso_categorical(P_cat, Q_cat, alpha_cat, w_mag)
    D_w_QR_iso = D_w_iso_categorical(Q_cat, R_cat, alpha_cat, w_mag)
    D_w_PR_iso = D_w_iso_categorical(P_cat, R_cat, alpha_cat, w_mag)
    eps_iso = abs(D_w_PR_iso - D_w_PQ_iso - D_w_QR_iso)
    epsilon_iso_cat.append(eps_iso)

epsilon_aniso_cat = np.array(epsilon_aniso_cat)
epsilon_iso_cat = np.array(epsilon_iso_cat)

print("Categorical Results:")
print(f"  ε_aniso(0) = {epsilon_aniso_cat[0]:.2e}")
print(f"  ε_aniso(max) = {epsilon_aniso_cat[-1]:.6f}")
print(f"  ε_iso(0) = {epsilon_iso_cat[0]:.2e}")
print(f"  ε_iso(max) = {epsilon_iso_cat[-1]:.2e}")

In [ ]:
# ============================================
# Categorical: Verify Third-Order Mixed Derivative
# ============================================

def verify_third_derivative_categorical(theta_q: np.ndarray, u: np.ndarray, v: np.ndarray,
                                         beta: float, gamma: float, w_mag: float) -> dict:
    """
    Compute the connection perturbation for categorical distribution.
    Returns key quantities for verification.
    """
    # h(θ_Q)
    v_dot_theta = np.dot(v, theta_q)
    h_val = 1.0 + gamma * v_dot_theta**2
    
    # ∂h/∂θ'
    dh = 2.0 * gamma * v_dot_theta * v
    
    # δΓ_{ij,k} = -β|w| · 2 u_i u_j (∂h/∂θ'_k)
    n = len(theta_q)
    delta_Gamma = np.zeros((n, n, n))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                delta_Gamma[i, j, k] = -beta * w_mag * 2 * u[i] * u[j] * dh[k]
    
    # Frobenius norm as a scalar measure
    frob_norm = np.linalg.norm(delta_Gamma.flatten())
    
    return {
        'h': h_val,
        'dh': dh,
        'v_dot_theta': v_dot_theta,
        'delta_Gamma': delta_Gamma,
        'frobenius_norm': frob_norm
    }

# Compute at each base point
print("Third-Order Mixed Derivative Verification (Categorical):")
print("")

for name, theta in [('θ_Q', theta_Q_cat), ('θ_R', theta_R_cat)]:
    result = verify_third_derivative_categorical(theta, u_cat, v_cat, beta_cat, gamma_cat, 1.0)
    print(f"At {name}:")
    print(f"  v^T θ = {result['v_dot_theta']:.4f}")
    print(f"  h(θ) = {result['h']:.4f}")
    print(f"  ||∂h/∂θ|| = {np.linalg.norm(result['dh']):.4f}")
    print(f"  ||δΓ||_F = {result['frobenius_norm']:.6f}")
    print("")

In [ ]:
# ============================================
# Categorical: Plot Results
# ============================================

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Main plot
ax1 = axes[0]
ax1.plot(w_values_cat, epsilon_aniso_cat, 'b-', linewidth=2, label='Anisotropic (point-dependent)')
ax1.plot(w_values_cat, epsilon_iso_cat, 'r--', linewidth=2, label='Isotropic')
ax1.set_xlabel('|w|', fontsize=12)
ax1.set_ylabel('Pythagorean residual ε(w)', fontsize=12)
ax1.set_title('Categorical: Pythagorean Residual vs |w|', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 1])
ax1.set_ylim(bottom=0)

# Right: Log scale
ax2 = axes[1]
ax2.semilogy(w_values_cat, epsilon_aniso_cat + 1e-20, 'b-', linewidth=2, label='Anisotropic')
ax2.semilogy(w_values_cat, epsilon_iso_cat + 1e-20, 'r--', linewidth=2, label='Isotropic')
ax2.axhline(y=1e-15, color='gray', linestyle=':', label='Machine precision')
ax2.set_xlabel('|w|', fontsize=12)
ax2.set_ylabel('ε(w) [log scale]', fontsize=12)
ax2.set_title('Categorical: Log Scale', fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_xlim([0, 1])

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/D_categorical_point_dependent.png', dpi=150, bbox_inches='tight')
plt.savefig(f'{SAVE_DIR}/D_categorical_point_dependent.pdf', bbox_inches='tight')
plt.show()

print(f"Figures saved to: {SAVE_DIR}/D_categorical_point_dependent.png/pdf")

## Part 3: Comparison and Summary

In [ ]:
# ============================================
# Cross-Distribution Comparison
# ============================================

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Anisotropic comparison
ax1 = axes[0]
ax1.plot(w_values / 2, epsilon_aniso_gauss, 'b-', linewidth=2, label='2D Gaussian')
ax1.plot(w_values_cat, epsilon_aniso_cat, 'g--', linewidth=2, label='Categorical')
ax1.set_xlabel('|w| (normalized)', fontsize=12)
ax1.set_ylabel('Pythagorean residual ε(w)', fontsize=12)
ax1.set_title('Anisotropic: Both Break', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: Isotropic comparison (log scale)
ax2 = axes[1]
ax2.semilogy(w_values / 2, epsilon_iso_gauss + 1e-20, 'b-', linewidth=2, label='2D Gaussian')
ax2.semilogy(w_values_cat, epsilon_iso_cat + 1e-20, 'g--', linewidth=2, label='Categorical')
ax2.axhline(y=1e-15, color='gray', linestyle=':', label='Machine precision')
ax2.set_xlabel('|w| (normalized)', fontsize=12)
ax2.set_ylabel('ε(w) [log scale]', fontsize=12)
ax2.set_title('Isotropic: Both Preserve', fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/D_comparison_point_dependent.png', dpi=150, bbox_inches='tight')
plt.savefig(f'{SAVE_DIR}/D_comparison_point_dependent.pdf', bbox_inches='tight')
plt.show()

print(f"Comparison figures saved to: {SAVE_DIR}/D_comparison_point_dependent.png/pdf")

In [ ]:
# ============================================
# Summary Table
# ============================================

print("="*70)
print("SUMMARY: Point-Dependent Directional Weighting Experiments")
print("="*70)
print("")
print("Perturbation form: D_w(P,Q) = D(P,Q) + β|w| h(θ_Q) (u^T(θ_P - θ_Q))²")
print("                   h(θ') = 1 + γ(v^T θ')²")
print("")
print("-"*70)
print(f"{'Distribution':<25} {'ε(0)':<15} {'ε_iso(max)':<15} {'ε_aniso(max)':<15}")
print("-"*70)
print(f"{'2D Gaussian':<25} {epsilon_aniso_gauss[0]:.2e}      {epsilon_iso_gauss[-1]:.2e}      {epsilon_aniso_gauss[-1]:.6f}")
print(f"{'Categorical (4-class)':<25} {epsilon_aniso_cat[0]:.2e}      {epsilon_iso_cat[-1]:.2e}      {epsilon_aniso_cat[-1]:.6f}")
print("-"*70)
print("")
print("Key findings:")
print("  ✓ Isotropic perturbations preserve Pythagorean relation (ε ≈ 0)")
print("  ✓ Anisotropic (point-dependent) perturbations break Pythagorean relation (ε > 0)")
print("  ✓ Connection perturbation δΓ ≠ 0 verified at non-degenerate points")
print("")
print("="*70)

In [ ]:
# ============================================
# Save Results to JSON
# ============================================

results = {
    'experiment': 'Paper D: Point-Dependent Directional Weighting',
    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'perturbation_form': 'D_w(P,Q) = D(P,Q) + β|w| h(θ_Q) (u^T(θ_P - θ_Q))², h(θ) = 1 + γ(v^T θ)²',
    'gaussian': {
        'triple': {
            'P': theta_P.tolist(),
            'Q': theta_Q.tolist(),
            'R': theta_R.tolist()
        },
        'parameters': {
            'u': u_gauss.tolist(),
            'v': v_gauss.tolist(),
            'beta': beta_gauss,
            'gamma': gamma_gauss,
            'alpha': alpha_gauss,
            'w_range': [float(w_values[0]), float(w_values[-1])]
        },
        'results': {
            'epsilon_baseline': float(epsilon_baseline),
            'epsilon_aniso_max': float(epsilon_aniso_gauss[-1]),
            'epsilon_iso_max': float(epsilon_iso_gauss[-1])
        }
    },
    'categorical': {
        'triple': {
            'P': P_cat.tolist(),
            'Q': Q_cat.tolist(),
            'R': R_cat.tolist()
        },
        'natural_parameters': {
            'theta_P': theta_P_cat.tolist(),
            'theta_Q': theta_Q_cat.tolist(),
            'theta_R': theta_R_cat.tolist()
        },
        'parameters': {
            'u': u_cat.tolist(),
            'v': v_cat.tolist(),
            'beta': beta_cat,
            'gamma': gamma_cat,
            'alpha': alpha_cat,
            'w_range': [float(w_values_cat[0]), float(w_values_cat[-1])]
        },
        'results': {
            'orthogonality_check': float(orthog_check),
            'epsilon_baseline': float(epsilon_baseline_cat),
            'epsilon_aniso_max': float(epsilon_aniso_cat[-1]),
            'epsilon_iso_max': float(epsilon_iso_cat[-1])
        }
    },
    'conclusion': {
        'isotropic_preserves': True,
        'anisotropic_breaks': True,
        'connection_perturbation_nonzero': True
    }
}

with open(f'{SAVE_DIR}/D_point_dependent_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to: {SAVE_DIR}/D_point_dependent_results.json")

In [ ]:
# ============================================
# Save CSV for detailed analysis
# ============================================

# Gaussian
with open(f'{SAVE_DIR}/D_gaussian_point_dependent.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['w_magnitude', 'epsilon_anisotropic', 'epsilon_isotropic'])
    for i, w in enumerate(w_values):
        writer.writerow([w, epsilon_aniso_gauss[i], epsilon_iso_gauss[i]])

# Categorical
with open(f'{SAVE_DIR}/D_categorical_point_dependent.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['w_magnitude', 'epsilon_anisotropic', 'epsilon_isotropic'])
    for i, w in enumerate(w_values_cat):
        writer.writerow([w, epsilon_aniso_cat[i], epsilon_iso_cat[i]])

print(f"CSV files saved to:")
print(f"  {SAVE_DIR}/D_gaussian_point_dependent.csv")
print(f"  {SAVE_DIR}/D_categorical_point_dependent.csv")

## Reproducibility Information

In [ ]:
print("="*70)
print("REPRODUCIBILITY INFORMATION")
print("="*70)
print(f"Random seed: 42")
print(f"NumPy version: {np.__version__}")
print(f"Execution completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("")
print(f"Output directory: {SAVE_DIR}")
print("")
print("Output files:")
print("  - D_gaussian_point_dependent.png/pdf")
print("  - D_categorical_point_dependent.png/pdf")
print("  - D_comparison_point_dependent.png/pdf")
print("  - D_point_dependent_results.json")
print("  - D_gaussian_point_dependent.csv")
print("  - D_categorical_point_dependent.csv")
print("="*70)